In [ ]:
import csv
import re
from pathlib import Path

import pandas as pd

pd.set_option("display.max_colwidth", None)

In [ ]:
SUBMISSIONS = [
    "RSNA_20220228",
    "RSNA_20220308",
    "RSNA_20220314",
]

In [ ]:
print(SUBMISSION)
PACKAGES_PATH = Path(f"/Users/andrewprokhorenkov/CTDS/projects/midrc/indexing-data/packages_rsna/packages_{SUBMISSION}")
PACKAGES_PATH.mkdir(parents=True, exist_ok=True)

In [ ]:
def process_submission(SUBMISSION):
    # useful paths for data manipulation
    print(SUBMISSION)
    PACKAGES_PATH = Path(f"/Users/andrewprokhorenkov/CTDS/projects/midrc/indexing-data/packages_rsna/packages_{SUBMISSION}")
    PACKAGES_PATH.mkdir(parents=True, exist_ok=True)

    RSNA_PATH = Path("/Users/andrewprokhorenkov/CTDS/projects/midrc/midrc_indexing_scripts/midrc-ricord-2021-08-20/image_file_manifest_RSNA_20210820.tsv")

    list_of_packages = []

    image_file_manifest = pd.read_csv(RSNA_PATH, sep="\t")

    rename_columns = {
        "case_ids": "case_id",
        "study_uid": "study_id",
        "series_uid": "series_id",
        "submitter_id": "instance_id",
    }

    image_file_manifest = image_file_manifest.rename(columns=rename_columns)
    image_file_manifest = image_file_manifest.drop(columns=["modality"])
    image_file_manifest["instance_id"] = image_file_manifest["instance_id"].apply(lambda v: v.split("_")[1])

    for _, row in image_file_manifest.iterrows():
        case_id = row["case_id"]
        study_id = row["study_id"]
        series_id = row["series_id"]
        # instance_id = row["instance_id"]

        series_path = f"./cases/{case_id}/{study_id}/{series_id}.tsv\n"
        if series_path not in list_of_packages:
            list_of_packages.append(series_path)

        folder = PACKAGES_PATH / "cases" / case_id / study_id

        folder.mkdir(parents=True, exist_ok=True)
        
        series_file = folder / f"{series_id}.tsv"
        series_file_exist = series_file.exists()

        with open(series_file, mode="a") as f:
            fieldnames = ["file_name", "file_size", "md5sum", "case_id", "study_id", "series_id", "instance_id", "storage_urls"]
            writer = csv.DictWriter(f, delimiter="\t", fieldnames=fieldnames)

            if not series_file_exist:
                writer.writeheader()
            writer.writerow(row.to_dict())

    with open(PACKAGES_PATH / "packages.txt", "w") as f:
        f.writelines(list_of_packages)

In [ ]:
for SUBMISSION in SUBMISSIONS:
    process_submission(SUBMISSION)